## Import Test Dataset

In [147]:
import numpy as np
import pandas as pd
import os
from scipy.stats import gmean
import simple_teximport as texi

In [148]:
# get the directory of each test data file
paths = []
dir = "test_data"
files = os.listdir(dir)
for file in files:
    paths.append(dir + '/' + file)

paths.sort()
paths.pop(0)
#paths.remove('Benchmark/.ipynb_checkpoints')
paths



['test_data/GSM7964198_CC2_13_FP_Y_1_S13_geneCOUNT.txt',
 'test_data/GSM7964199_CC2_14_FP_Y_2_S14_geneCOUNT.txt',
 'test_data/GSM7964200_CC2_15_FP_Y_3_S15_geneCOUNT.txt',
 'test_data/GSM7964201_CC2_16_FP_Y_4_S16_geneCOUNT.txt',
 'test_data/GSM7964202_CC2_17_FP_O_1_S17_geneCOUNT.txt',
 'test_data/GSM7964203_CC2_18_FP_O_2_S18_geneCOUNT.txt',
 'test_data/GSM7964204_CC2_19_FP_O_3_S19_geneCOUNT.txt',
 'test_data/GSM7964205_CC2_20_FP_O_4_S20_geneCOUNT.txt']

In [149]:
# load the raw counts with simple_teximport
column_names = ['young1', 'young2', 'young3','young4', 'old1', 'old2', 'old3', 'old4']
test_txi = texi.simple_texi(paths, column_names, 0, 1, None, None)
test_txi.counts

,Tracking_ID,young1,young2,young3,young4,old1,old2,old3,old4
0,ENSMUSG00000102693,0,0,0,0,0,0,0,0
1,ENSMUSG00000064842,0,0,0,0,0,0,0,0
2,ENSMUSG00000051951,11,7,9,10,10,14,13,17
3,ENSMUSG00000102851,0,0,0,0,0,0,0,0
4,ENSMUSG00000103377,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
51821,ENSMUSG00000095134,302,227,564,629,696,369,538,462
51822,ENSMUSG00000095366,14,4,13,40,17,17,24,18
51823,ENSMUSG00000096768,3269,1194,2747,5018,2889,2734,3301,8459
51824,ENSMUSG00000099871,81,88,147,148,151,78,100,96


In [260]:
#filter genes with counts less than or equal to 10
test_txi.filter(10,2)
test_txi.counts

,Tracking_ID,young1,young2,young3,young4,old1,old2,old3,old4
2,ENSMUSG00000051951,11,7,9,10,10,14,13,17
16,ENSMUSG00000025900,4,8,5,2,28,46,53,35
19,ENSMUSG00000025902,129,95,100,134,250,175,196,233
30,ENSMUSG00000103922,43,41,88,78,59,32,63,55
31,ENSMUSG00000033845,1258,1011,1274,1386,1499,1378,1493,1384
...,...,...,...,...,...,...,...,...,...
51821,ENSMUSG00000095134,302,227,564,629,696,369,538,462
51822,ENSMUSG00000095366,14,4,13,40,17,17,24,18
51823,ENSMUSG00000096768,3269,1194,2747,5018,2889,2734,3301,8459
51824,ENSMUSG00000099871,81,88,147,148,151,78,100,96


## Normalize the raw counts

In [261]:
#helper method, outputs a list of gene ids
def get_genes(test_txi):
    columns = test_txi.counts.columns
    genes = test_txi.counts[columns[0]].unique()
    return genes

In [257]:
from scipy.stats import gmean
def normalize(test_txi):
    """ Function to normalize read counts for each gene
    
    Parameters
    ----------
    test_txi: dataframe with int and string
       Table generated by teximport that combined raw counts from datasets 
    gene_lengths: list of int
       List of gene lengths for each gene. e.g. [length1, length2, ...]
    
    Returns
    -------
    normalized_data:
        dataframe with normalized read counts
    """
    columns = test_txi.counts.columns
    raw_data = test_txi.counts.set_index(columns[0])
    geometric_means = gmean(raw_data, axis=1)
    size_factors = raw_data.div(geometric_means[:, None], axis=0).replace(np.inf, np.nan).median(axis=0)
    normalized_data = raw_data / size_factors

    return normalized_data

    

In [258]:
normalized_counts = normalize(test_txi)
normalized_counts

,young1,young2,young3,young4,old1,old2,old3,old4
Tracking_ID,,,,,,,,
ENSMUSG00000051951,11.934432,8.503578,10.056581,9.976892,9.138304,12.963272,12.148356,14.121419
ENSMUSG00000025900,4.339793,9.718375,5.586989,1.995378,25.587252,42.593608,49.527915,29.073510
ENSMUSG00000025902,139.958335,115.405700,111.739789,133.690357,228.457603,162.040901,183.159836,193.546511
ENSMUSG00000103922,46.652778,49.806671,98.331014,77.819760,53.915994,29.630336,58.872804,45.686945
ENSMUSG00000033845,1364.865004,1228.159610,1423.564911,1382.797277,1369.831790,1275.956356,1395.192014,1149.649662
...,...,...,...,...,...,...,...,...
ENSMUSG00000095134,327.654397,275.758884,630.212410,627.546528,636.025968,341.674815,502.755059,383.770335
ENSMUSG00000095366,15.189277,4.859187,14.526173,39.907569,15.535117,15.741116,22.427735,14.952091
ENSMUSG00000096768,3546.696104,1450.467433,3069.492002,5006.404573,2640.056065,2531.541855,3084.748051,7026.652090


## Calculate the Log2 Fold Change

In [264]:
#basic log2 fold change
def log2_fc(conditions, test_txi):
    """ Function to compute log2 fold change for each gene
    
    Parameters
    ----------
    test_txi: dataframe with int and string
       Table generated by teximport that combined raw counts from datasets 
    gene_lengths: list of int
       List of gene lengths for each gene. e.g. [length1, length2, ...]
    
    Returns
    -------
    normalized_data:
        dataframe with normalized read counts
    """
    columns = test_txi.counts.columns
    genes = test_txi.counts[columns[0]]

    normalized_counts = normalize(test_txi)

    data = normalized_counts.rename(columns=lambda x: "")
    data = data.values

    
    # add a pseudocount to avoid log2 of zero
    pseudocount = 1
    data += pseudocount

    treated_mean = np.mean(data[:, conditions == 1], axis=1)
    control_mean = np.mean(data[:, conditions == 0], axis=1)

    log2fc = np.log2(treated_mean / control_mean)

    log2fc_df = pd.DataFrame({
        columns[0]: genes,
        'Log2FoldChange': log2fc
    })
    log2fc_df = log2fc_df.set_index(columns[0])

    return log2fc_df

In [263]:
conditions = np.array([0, 0, 0, 0, 1, 1, 1, 1])
log2fc = log2_fc(conditions, test_txi)
log2fc

,Log2FoldChange
Tracking_ID,
ENSMUSG00000051951,0.235897
ENSMUSG00000025900,2.555969
ENSMUSG00000025902,0.611419
ENSMUSG00000103922,-0.525951
ENSMUSG00000033845,-0.056843
...,...
ENSMUSG00000095134,0.002360
ENSMUSG00000095366,-0.111282
ENSMUSG00000096768,0.225267


## Finding Standard Error of Each Gene

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [235]:
def dispersion_helper(df):
    mean = df['count'].mean()
    variance = df['count'].var()  # using variance as a proxy for dispersion
    dispersion = (variance - mean) / (mean ** 2)
    if dispersion < 0:
        dispersion = np.nan
    return mean, dispersion

def estimate_dispersion(test_txi):

    columns = test_txi.counts.columns
    normalized_counts = normalize(test_txi)

    # adjust the format of the dataframe
    counts_long = normalized_counts.reset_index().melt(id_vars=columns[0], var_name='sample', value_name='count')
    
    results = []
    genes = get_genes(test_txi)
    for gene in genes:
        # get the read counts for current gene
        df_gene = counts_long[counts_long[columns[0]] == gene]
        
        # use helper to get estimated dispersion
        mean, dispersion = dispersion_helper(df_gene)
        
        # append the results
        results.append({columns[0]: gene, 'dispersion': dispersion})
    
    results_df = pd.DataFrame(results)
    results_df = results_df.set_index(columns[0])
    return results_df

In [236]:
dispersions = estimate_dispersion(test_txi)
dispersions

,dispersion
Tracking_ID,
ENSMUSG00000051951,NaN
ENSMUSG00000025900,0.716355
ENSMUSG00000025902,0.060484
ENSMUSG00000103922,0.119949
ENSMUSG00000033845,0.004470
...,...
ENSMUSG00000095134,0.103831
ENSMUSG00000095366,0.262089
ENSMUSG00000096768,0.237693


In [259]:


result = []
genes = get_genes(test_txi)
for gene in genes:
    fc = log2fc.loc[gene, 'Log2FoldChange']
    se = dispersions.loc[gene, 'dispersion']
    stat = fc/se
    result.append({columns[0]: gene, 'Stats': stat})

result = pd.DataFrame(result)
result = result.set_index(columns[0])

result

    

,Stats
Tracking_ID,
ENSMUSG00000051951,NaN
ENSMUSG00000025900,3.568021
ENSMUSG00000025902,10.108794
ENSMUSG00000103922,-4.384796
ENSMUSG00000033845,-12.715661
...,...
ENSMUSG00000095134,0.022732
ENSMUSG00000095366,-0.424599
ENSMUSG00000096768,0.947723


In [265]:
# INCOMPLETE
# finding the estimate dispersion parameter as a function of the mean using a negative binomial distribution 

def negative_binomial_dispersion(test_txi):
    columns = test_txi.counts.columns
    normalized_counts = normalize(test_txi)

    
    counts_long = normalized_counts.reset_index().melt(id_vars=columns[0], var_name='sample', value_name='read_count')
    
    results = []
    genes = counts_long[columns[0]].unique()
    for gene in genes:
        df_gene = counts_long[counts_long[columns[0]] == gene]
        
        # use the negative binomial model
        model = smf.glm(formula='read_count ~ 1', data=df_gene, family=sm.families.NegativeBinomial())
        result = model.fit()
        
        # extract the dispersion parameter
        dispersion = result.scale
        mean_read_count = df_gene['read_count'].mean()
        
        # Append the results
        results.append({columns[0]: gene, 'mean': mean_read_count, 'dispersion': dispersion})

    results_df = pd.DataFrame(results)

    return results_df



In [195]:
negative_binomial_dispersion(test_txi)

,Tracking_ID,mean,dispersion
0,ENSMUSG00000051951,11.105354,1.0
1,ENSMUSG00000025900,21.052853,1.0
2,ENSMUSG00000025902,158.499879,1.0
3,ENSMUSG00000103922,57.589538,1.0
4,ENSMUSG00000033845,1323.752078,1.0
...,...,...,...
18517,ENSMUSG00000095134,465.674799,1.0
18518,ENSMUSG00000095366,17.892283,1.0
18519,ENSMUSG00000096768,3544.507272,1.0
18520,ENSMUSG00000099871,111.263019,1.0
